In [2]:
import pandas as pd 
import numpy as np

In [5]:
df_movies = pd.read_csv('dados02 - movie_lens/movies.csv', sep=',')

In [6]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [8]:
df_rating = pd.read_csv('dados02 - movie_lens/ratings.csv', sep=',')

In [9]:
df_rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


### Quantos gêneros "The Matrix" tem? Quais são os gêneros? 

In [11]:
#Para achar quais filmes possuem Matrix no título
mask_title = df_movies.title.str.contains('Matrix')
df_movies[mask_title]

,movieId,title,genres
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4351,6365,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
4639,6934,"Matrix Revolutions, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX


In [12]:
matrix = df_movies.query('movieId == 2571')

In [13]:
n_generos = matrix.genres.str.split('|').to_list()[0]
print(f"Os gêneros do filme são {str(n_generos)[1:-1]}")


Os gêneros do filme são 'Action', 'Sci-Fi', 'Thriller'


### Quantos gêneros cada filme tem, em média?

In [14]:
def encontra_qtd_generos(df: pd.DataFrame, col:str) -> list:
    generos_filme = df[col].str.split('|').to_list()
    n_generos = [len(generos) for generos in generos_filme]
    return n_generos

In [15]:
df_movies['n_generos'] = encontra_qtd_generos(df_movies, 'genres')

In [16]:
print(f"Cada filme tem em média {df_movies['n_generos'].mean().round(2)} gêneros")

Cada filme tem em média 2.27 gêneros


### Quais são os cinco filmes com maior número de gêneros?

In [22]:
top_5 = df_movies.nlargest(5,'n_generos')
print(f' Os cinco filmes com maior número de gêneros são: {str(top_5.title.to_list())[1:-1]}')

 Os cinco filmes com maior número de gêneros são: 'Rubber (2010)', 'Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)', 'Mulan (1998)', 'Who Framed Roger Rabbit? (1988)', 'Osmosis Jones (2001)'


### Quantos usuários deram nota para o filme "The Matrix"?

In [37]:
matrix_id = 2571
reviews = df_rating.query('movieId == @matrix_id')

print(f"{len(reviews)} usuários pontuaram 'The Matrix'.")

278 usuários pontuaram 'The Matrix'.


### Quantos usuários, em média, deram nota para cada filme?

In [41]:
print(f"Em média, {round(len(df_rating)/len(df_movies),2)} usuários avaliaram cada filme")

Em média, 10.35 usuários avaliaram cada filme


In [50]:
resposta = df_rating[['movieId', 'rating']].groupby('movieId').count()
# print((resposta['rating'].sum()/len(resposta)).round(2))

### Quais são os cinco filmes para os quais mais usuários deram nota?

In [54]:
top_avaliacoes = resposta.nlargest(5,'rating')

In [55]:
top_filmes= top_avaliacoes.merge(df_movies, how='left', on='movieId')
top_filmes = top_filmes[['rating','title']]
top_filmes.rename(columns={'rating':'reviews'})

,reviews,title
0,329,Forrest Gump (1994)
1,317,"Shawshank Redemption, The (1994)"
2,307,Pulp Fiction (1994)
3,279,"Silence of the Lambs, The (1991)"
4,278,"Matrix, The (1999)"


 ### Qual o rating médio do filme "The Matrix"?

In [56]:
avalicaoes = df_rating.query('movieId == @matrix_id') 
print(f"A pontuação média do filme The Matrix é {avalicaoes['rating'].mean():.2f}")

A pontuação média do filme The Matrix é 4.19


### Cada filme tem um rating médio. Qual a média desses ratings, por gênero?

In [57]:
def encontra_generos(dataframe: pd.DataFrame, col: 'str') -> list:
    generos = dataframe[col].str.split('|').to_list()
    return generos

In [59]:
df_filme_test = df_movies.copy()

df_filme_test['genres'] = encontra_generos(df_movies, 'genres')
df_filme_test = df_filme_test.explode('genres',ignore_index=False)
df_filme_test

,movieId,title,genres,n_genres,n_generos
0,1,Toy Story (1995),Adventure,5,5
0,1,Toy Story (1995),Animation,5,5
0,1,Toy Story (1995),Children,5,5
0,1,Toy Story (1995),Comedy,5,5
0,1,Toy Story (1995),Fantasy,5,5
...,...,...,...,...,...
9738,193583,No Game No Life: Zero (2017),Fantasy,3,3
9739,193585,Flint (2017),Drama,1,1
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action,2,2
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Animation,2,2


In [60]:
df_merge = df_filme_test.merge(df_rating, how='left', on='movieId')
df_merge.groupby(['genres']).agg({'rating':'mean'})

,rating
genres,
(no genres listed),3.489362
Action,3.447984
Adventure,3.508609
Animation,3.629937
Children,3.412956
Comedy,3.384721
Crime,3.658294
Documentary,3.797785
Drama,3.656184


### Para cada gênero de filme, quantos usuários deram cada nota possível?

In [61]:
df_avaliacao_genero = df_merge.groupby(['genres','rating']).agg({'rating':'count'})

In [62]:
df_avaliacao_genero

rating
genres             rating        
(no genres listed) 0.5          2
                   1.0          2
                   2.0          2
                   2.5          6
                   3.0          6
...                           ...
Western            3.0        442
                   3.5        201
                   4.0        543
                   4.5        158
                   5.0        295

[199 rows x 1 columns]